In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set up paths
base_dir = "C:/Photos"

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 20% for validation

# Load training data
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',  # 'binary' because it's a 2-class problem (GO/NG)
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the trained model
model.save("fan_model.h5")


Found 270 images belonging to 2 classes.
Found 66 images belonging to 2 classes.


c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\teyin\miniconda3\envs\ForML\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 379ms/step - accuracy: 0.5123 - loss: 0.9083 - val_accuracy: 0.6212 - val_loss: 0.6805
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 249ms/step - accuracy: 0.6154 - loss: 0.6565 - val_accuracy: 0.6212 - val_loss: 0.6636
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.6178 - loss: 0.6288 - val_accuracy: 0.6212 - val_loss: 0.6555
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 244ms/step - accuracy: 0.7038 - loss: 0.5353 - val_accuracy: 0.3939 - val_loss: 0.7148
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 246ms/step - accuracy: 0.8053 - loss: 0.4360 - val_accuracy: 0.4394 - val_loss: 0.6793
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 244ms/step - accuracy: 0.9450 - loss: 0.2675 - val_accuracy: 0.6515 - val_loss: 0.5582
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 241ms/step - accuracy: 0.9795 - loss: 0.1489 - val_accuracy: 0.8485 - val_loss: 0.3735
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 243ms/step - accuracy: 0.9568 - loss: 0.1084 - val_accuracy: 0.8636 - val_loss:

In [2]:
import cv2
import numpy as np
import tensorflow as tf
import glob
import os

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image at {image_path}")
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    prediction = model.predict(img)
    
    # Handle cases where the prediction might not be well-formed
    if prediction is not None and len(prediction) > 0:
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold for binary classification
        confidence = prediction[0][0] * 100  # Confidence score in percentage
        return class_idx, confidence
    else:
        return None, None

# Function to classify all images in given directories
def classify_images_in_directories(directories):
    for directory_path in directories:
        print(f"\nClassifying images in directory: {directory_path}")
        
        # Use glob to find all image files (supports .jpg, .png, .jpeg)
        image_files = glob.glob(os.path.join(directory_path, "*.jpg")) + \
                      glob.glob(os.path.join(directory_path, "*.png")) + \
                      glob.glob(os.path.join(directory_path, "*.jpeg"))
        
        if not image_files:
            print("No images found in the directory!")
            continue
        
        for image_path in image_files:
            result, confidence = classify_image(image_path)
            if result is not None:
                if result == 1:
                    print(f"GO (Correctly Aligned) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
                else:
                    print(f"NG (Defective) with {confidence:.2f}% confidence for image: {os.path.basename(image_path)}")
            else:
                print(f"Unable to classify image: {os.path.basename(image_path)}")

# Example usage: Classify images from both "60" and "100" folders
directories = ["C:/Photos/60", "C:/Photos/100"]
classify_images_in_directories(directories)



Classifying images in directory: C:/Photos/60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
NG (Defective) with 44.81% confidence for image: image-30-B20008.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
GO (Correctly Aligned) with 92.05% confidence for image: image-30-B20009.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
GO (Correctly Aligned) with 93.85% confidence for image: image-30-B20023.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
GO (Correctly Aligned) with 91.79% confidence for image: image-30-B20024.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
GO (Correctly Aligned) with 93.03% confidence for image: image-30-B20025.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
GO (Correctly Aligned) with 90.99% confidence for image: image-30-B20026.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
GO (Correctly Aligned) with 79.52% confidence for image: image-30-B20027.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
GO (Correctly Aligned) with 81.11% confidence for image: image-30-B20028.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
GO 

### The code needs to import classify function

In [17]:
import cv2
import numpy as np
import tensorflow as tf
import gradio as gr
from PIL import Image

# Load the pre-trained model
model = tf.keras.models.load_model("fan_model.h5")

# Recompile the model to suppress the warning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to classify a single image
def classify_image(image):
    # Convert the image to a format suitable for OpenCV
    img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (80, 80))
    img = img.reshape(1, 80, 80, 3) / 255.0  # Normalize
    
    # Make prediction
    prediction = model.predict(img)
    
    # Classification logic
    if prediction is not None and len(prediction) > 0:
        # Set threshold for GO/NG classification
        # Adjusted based on expected results for the provided images
        class_idx = 1 if prediction[0] >= 0.5 else 0  # Threshold set to 0.6
        # confidence = prediction[0][0] * 100  # Confidence score in percentage
        
        if class_idx == 1:
            result_text = f"<div style='color: red; font-size: 24px; font-weight: bold;'>NG</div>"
            return result_text
        else:
            result_text = f"<div style='color: green; font-size: 24px; font-weight: bold;'>GO</div>"
            return result_text
        
    else:
        return "<div style='color: black; font-size: 20px;'>Unable to classify image</div>"

# Create Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.HTML(label="Classification Result"),
    title="Cooling Fan GO/NG Inspection",
    description="Upload an image of a cooling fan to check if it is correctly aligned (GO) or defective (NG).",
)

# Launch the interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
